In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data import ModelNet40,download,load_data
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from matplotlib import pyplot as plt
import pickle
import warnings
import math
import time
from datetime import datetime
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torch import jit
%matplotlib inline
import os
from torch.utils.data import Dataset, TensorDataset

In [2]:
train_losses = []
test_results_exp = []
test_accs = []

In [3]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lrt = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrt

In [4]:
device = torch.device("cuda:2")

In [30]:
data = ModelNet40(num_points=2048, partition='train', gaussian_noise=False,
                       unseen=False, factor=4)
data_test = ModelNet40(num_points=2048, partition='test', gaussian_noise=False,
                       unseen=False, factor=4)
train_loader = DataLoader(data, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(data_test, batch_size=128, shuffle=True, drop_last=True)
rl_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

/home/gwu/ConvexICP/data.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [6]:
def train_eval_icp(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
    for batch_idx, (src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba) in enumerate(train_loader):
        src = src.to(device)
        target = target.to(device)
        rotation_ab = rotation_ab.to(device)
        translation_ab = translation_ab.to(device)
        rotation_ba = rotation_ba.to(device)
        translation_ba = translation_ba.to(device)
        optimizer.zero_grad()
        output_train = model(src,target,rotation_ab,translation_ab)
        loss = criterion(output_train, target)
        loss.backward()
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
        train_error_logs.append(loss.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    
    model.eval()
    test_loss = 0
    correct_test = 0
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in test_loader:
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target).item()*len(src)  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    return train_error_logs,test_loss_log,test_acc_log

In [18]:
def cal_rt_loss(ra,ta,rb,tb):
    r_loss = F.mse_loss(ra,rb).item()
    t_loss = F.mse_loss(ta,tb).item()
    return r_loss, t_loss

In [7]:
class PointNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, emb_dims, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(emb_dims)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        return x

In [8]:
class CPNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(CPNet, self).__init__()
        self.base_net = PointNet()
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2, R, T):
        
        trans = torch.matmul(R, x1) + T.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [9]:
torch.Tensor(1,3,3)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [24]:
class CPEvalNet(nn.Module):
    def __init__(self, base_net, emb_dims=512):
        super(CPEvalNet, self).__init__()
        self.base_net = PointNet()
        self.base_net.load_state_dict(base_net.state_dict())
        for param in self.base_net.parameters():
            param.require_grad = False
        self.rotation = nn.Parameter(torch.Tensor(1,3,3))
        self.translation = nn.Parameter(torch.Tensor(1,3))
        nn.init.kaiming_uniform_(self.rotation, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.translation, a=math.sqrt(5))
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2):
        
        trans = torch.matmul(self.rotation, x1) + self.translation.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)

        return dist

In [11]:
def dummy_loss(output, target):
    loss = torch.mean(output)
    return loss

In [12]:
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [13]:
model = CPNet().to(device)

In [14]:
criterion = dummy_loss
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [54]:
train_time= []
train_loss = []
train_losses.append(train_loss)
tests = []
test_acc = []
test_results_exp.append(tests)
test_accs.append(test_acc)
ratio = 0
for epoch in range(1,600):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    train_error,test_error,test_acc_this = train_eval_icp(model,device, train_loader, test_loader, optimizer,criterion,epoch,'on')
    train_loss.extend(train_error)
    tests.extend(test_error)
    test_acc.extend(test_acc_this)
    train_time.append((datetime.now()-t1).total_seconds())
    print((datetime.now()-t1).total_seconds())

Train Epoch: 1 [0/9840 (0%)]	Loss: 0.000088	Time: 0.54
Train Epoch: 1 [2560/9840 (26%)]	Loss: 0.000006	Time: 9.78
Train Epoch: 1 [5120/9840 (53%)]	Loss: 0.000062	Time: 19.06
Train Epoch: 1 [7680/9840 (79%)]	Loss: 0.000014	Time: 28.31
Train Epoch: 1 Accuracy: 0/9840 (0.00%)

Test set: Average loss: 0.00004385, Accuracy: 0/2468 (0.00%)

40.846422
Train Epoch: 2 [0/9840 (0%)]	Loss: 0.000008	Time: 0.46


KeyboardInterrupt: 

In [40]:
def rt_learning(model, device, src, target, rotation_ab, translation_ab, optimizer,criterion,epoch,n_batch = 40):
    r_losses = []
    t_losses = []
    for _ in range(n_batch):
        model.train()
        t1 = datetime.now()
        src = src.to(device)
        target = target.to(device)
        rotation_ab = rotation_ab.to(device)
        translation_ab = translation_ab.to(device)
        optimizer.zero_grad()
        output_train = model(src,target)
        loss = criterion(output_train, target)
        loss.backward()
        optimizer.step()
        r_loss,t_loss = cal_rt_loss(model.rotation, model.translation,rotation_ab, translation_ab)
        r_losses.append(r_loss)
        t_losses.append(t_loss)
    return r_losses, t_losses

In [37]:
r_losses = []
t_losses = []

In [83]:
model_eval = CPEvalNet(model.base_net).to(device)

In [100]:
criterion = dummy_loss
lr_rl = 1e10
optimizer = torch.optim.SGD(model_eval.parameters(), lr_rl,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [85]:
for src_rl, target_rl, rotation_ab_rl, translation_ab_rl, rotation_ba_rl, translation_ba_rl,_,_ in rl_loader:
    break

In [102]:
train_time= []
r_loss = []
t_loss = []
r_losses.append(r_loss)
t_losses.append(t_loss)
ratio = 0
for epoch in range(1,600):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    r_batch_loss, t_batch_loss = rt_learning(model_eval, device, src_rl, target_rl, rotation_ab_rl, translation_ab_rl, 
                                     optimizer,criterion,epoch,n_batch=40)
    r_loss.extend(r_batch_loss)
    t_loss.extend(t_batch_loss)
    train_time.append((datetime.now()-t1).total_seconds())

    print('Train Epoch: {} \tRotation_Loss: {:.6f}\tTranslation_Loss: {:.6f}\tTime: {:.2f}'.format(
        epoch, r_batch_loss[-1],t_batch_loss[-1],(datetime.now()-t1).total_seconds()))

Train Epoch: 1 	Rotation_Loss: 0.456901	Translation_Loss: 0.005491	Time: 1.00
Train Epoch: 2 	Rotation_Loss: 0.456847	Translation_Loss: 0.005488	Time: 0.98
Train Epoch: 3 	Rotation_Loss: 0.456792	Translation_Loss: 0.005485	Time: 0.99
Train Epoch: 4 	Rotation_Loss: 0.456738	Translation_Loss: 0.005482	Time: 0.98
Train Epoch: 5 	Rotation_Loss: 0.456684	Translation_Loss: 0.005479	Time: 0.98
Train Epoch: 6 	Rotation_Loss: 0.456631	Translation_Loss: 0.005476	Time: 0.99
Train Epoch: 7 	Rotation_Loss: 0.456578	Translation_Loss: 0.005473	Time: 0.99
Train Epoch: 8 	Rotation_Loss: 0.456524	Translation_Loss: 0.005471	Time: 1.01
Train Epoch: 9 	Rotation_Loss: 0.456471	Translation_Loss: 0.005468	Time: 0.99
Train Epoch: 10 	Rotation_Loss: 0.456418	Translation_Loss: 0.005465	Time: 0.97
Train Epoch: 11 	Rotation_Loss: 0.456366	Translation_Loss: 0.005462	Time: 1.01
Train Epoch: 12 	Rotation_Loss: 0.456314	Translation_Loss: 0.005459	Time: 0.99
Train Epoch: 13 	Rotation_Loss: 0.456262	Translation_Loss: 0.

KeyboardInterrupt: 

In [103]:
model_eval.rotation

Parameter containing:
tensor([[[-0.1170,  0.7892,  0.4633],
         [ 0.2884, -0.0550,  0.0122],
         [-0.3263,  0.0933,  0.3119]]], device='cuda:2', requires_grad=True)

In [98]:
model_eval.rotation

Parameter containing:
tensor([[[-0.1176,  0.7919,  0.4648],
         [ 0.2894, -0.0554,  0.0123],
         [-0.3272,  0.0937,  0.3132]]], device='cuda:2', requires_grad=True)

In [99]:
rotation_ab_rl

tensor([[[ 0.8242, -0.5311,  0.1967],
         [ 0.5662,  0.7817, -0.2614],
         [-0.0149,  0.3268,  0.9450]]])